# Database Connection and Setup

This notebook is dedicated to setting up the environment, loading configuration files, and testing connectivity to your database.

## What this notebook does:
- Loads a YAML configuration file.
- Constructs a database connection string based on your configuration.
- Tests the connection to the database using SQLAlchemy.

In [2]:
import yaml

possible_config_paths = [
    "../../config/config.yaml",
]

config_path = None
for path in possible_config_paths:
    if os.path.exists(path):
        config_path = path
        print(f"Found config file at: {path}")
        break

if not config_path:
    raise FileNotFoundError(f"Config file not found. Tried these paths: {possible_config_paths}")

with open(config_path, "r") as f:
    config = yaml.safe_load(f)

print("Config loaded successfully")

Found config file at: ../../config/config.yaml
Config loaded successfully


In [ ]:
from sqlalchemy import create_engine, text

if 'database' in config and 'connection_string' in config['database']:
    db_uri = config['database']['connection_string']
    print(f"Using connection string from config: {db_uri}")
else:
    db_config = config.get('database', {})
    username = db_config.get('username', '')
    password = db_config.get('password', '')
    host = db_config.get('host', 'localhost')
    port = db_config.get('port', 5432)
    dbname = db_config.get('dbname', 'postgres')
    sslmode = db_config.get('sslmode', 'prefer')
    
    db_uri = f"postgresql://{username}:{password}@{host}:{port}/{dbname}?sslmode={sslmode}"
    print(f"Built connection string: {db_uri}")

In [ ]:
from sqlalchemy import create_engine, text

try:
    print(f"Using connection string from config: {db_uri}")
    engine = create_engine(db_uri)
    print("Engine created successfully")
    
    # Save the connection string for later use in other notebooks
    %store db_uri
    
    with engine.connect() as connection:
        result = connection.execute(text("SELECT 1 as test"))
        test_output = result.scalar()
        print(f"Test query executed successfully: {test_output}")
        
        result = connection.execute(text("SELECT table_name FROM information_schema.tables WHERE table_schema='public'"))
        tables = [row[0] for row in result]
        print(f"Tables in the database: {tables}")

        print("Engine created!")

except Exception as e:
    print(f"Error connecting to the database: {e}")

## Next Steps

Once this notebook executes without errors, your environment is correctly configured and connected to the database. You can then proceed to use this setup for further analysis in subsequent notebooks.